## 作業
在鐵達尼資料集中，今天我們專注觀察變數之間的相關性，以Titanic_train.csv 中，首先將有遺失值的數值刪除，並回答下列問題。
* Q1: 透過數值法計算  Age 和  Survived  是否有相關性?
* Q2:透過數值法計算  Sex 和 Survived  是否有相關性?
* Q3: 透過數值法計算  Age 和  Fare 是否有相關性?   
* 提示:    
1.產稱一個新的變數 Survived_cate ，資料型態傳換成類別型態      
2.把題目中的 Survived 用 Survived_cate 來做分析       
3.首先觀察一下這些變數的資料型態後，再來想要以哪一種判斷倆倆的相關性。     


In [3]:
# !pip install pingouin

  Using cached pingouin-0.3.10.tar.gz (204 kB)
  Using cached pandas_flavor-0.2.0-py2.py3-none-any.whl (6.6 kB)
  Using cached outdated-0.2.0.tar.gz (4.0 kB)
  Using cached tabulate-0.8.9-py3-none-any.whl (25 kB)
  Using cached xarray-0.16.2-py3-none-any.whl (736 kB)
  Using cached littleutils-0.2.2.tar.gz (6.6 kB)
  Using cached requests-2.25.1-py2.py3-none-any.whl (61 kB)
  Using cached chardet-4.0.0-py2.py3-none-any.whl (178 kB)
  Using cached idna-2.10-py2.py3-none-any.whl (58 kB)
  Using cached urllib3-1.26.3-py2.py3-none-any.whl (137 kB)
  Using cached threadpoolctl-2.1.0-py3-none-any.whl (12 kB)
  Using cached joblib-1.0.1-py3-none-any.whl (303 kB)
  Created wheel for pingouin: filename=pingouin-0.3.10-py3-none-any.whl size=203396 sha256=27ad2f0d8119d7b55b3f222cf1b6a920b88dfb2fa2073dab0ca9d404dc3716cf
  Stored in directory: c:\users\apply\appdata\local\pip\cache\wheels\b8\92\fa\0257fec4f00e04a8801d3d5c37b2492f969a231137e376bd1f
  Created wheel for outdated: filename=outdated-0.2

In [1]:
# conda list

# packages in environment at C:\Users\apply\anaconda3\envs\datascience3.6:
#
# Name                    Version                   Build  Channel
argon2-cffi               20.1.0           py37h2bbff1b_1  
async_generator           1.10             py37h28b3542_0  
attrs                     20.3.0             pyhd3eb1b0_0  
backcall                  0.2.0              pyhd3eb1b0_0  
blas                      1.0                         mkl  
bleach                    3.3.0              pyhd3eb1b0_0  
ca-certificates           2021.1.19            haa95532_0  
certifi                   2020.12.5        py37haa95532_0  
cffi                      1.14.5           py37hcd4344a_0  
colorama                  0.4.4              pyhd3eb1b0_0  
cycler                    0.10.0                   py37_0  
decorator                 4.4.2              pyhd3eb1b0_0  
defusedxml                0.6.0              pyhd3eb1b0_0  
entrypoints               0.3                      py37_0  
freetype        

In [4]:
# import library
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
import math
import statistics
import seaborn as sns
from IPython.display import display

import pingouin as pg
import researchpy   
%matplotlib inline

## 讀入資料

In [6]:
df_train = pd.read_csv("Titanic_train.csv")
print(df_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None


In [8]:
## 這邊我們做一個調整，把 Survived 變成離散型變數 Survived_cate
data = df_train.dropna()

from sklearn import preprocessing
#下面兩行程式，讓 Male=1, FeMale-0.
le = preprocessing.LabelEncoder()
data['Survived_cate'] = le.fit_transform(data['Survived'])
display(data)

C:\Users\apply\anaconda3\envs\datascience3.6\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived_cate
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S,0
10,11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,PP 9549,16.7000,G6,S,1
11,12,1,1,"Bonnell, Miss. Elizabeth",female,58.0,0,0,113783,26.5500,C103,S,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
871,872,1,1,"Beckwith, Mrs. Richard Leonard (Sallie Monypeny)",female,47.0,1,1,11751,52.5542,D35,S,1
872,873,0,1,"Carlsson, Mr. Frans Olof",male,33.0,0,0,695,5.0000,B51 B53 B55,S,0
879,880,1,1,"Potter, Mrs. Thomas Jr (Lily Alexenia Wilson)",female,56.0,0,1,11767,83.1583,C50,C,1
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,1


###  Q1: 透過數值法計算  Age 和  Survived  是否有相關性?


### Q2:透過數值法計算  Sex 和 Survived  是否有相關性?


### Q3: 透過數值法計算  Age 和  Fare 是否有相關性?  